1) Wealth Management: Wealth Managers get a one time wealth cut from the HNI clients and charge fees for each of the services they provide. The revenue will be the sum of the wealth cut and fees. Calculate how much each HNI has paid to the firm.

In [27]:
with total_fees as (
select HNI_id, sum(fee) as total_fees_collected
from dma.WMprovision
GROUP by HNI_id)
select wc.HNI_id, c.name, (tf.total_fees_collected + wc.wealthcut) as
total_hni_pay
from dma.wealthcut wc join total_fees tf on wc.HNI_id=tf.HNI_id join dma.client c on wc.HNI_id = c.ID;

(5 rows affected)

Total execution time: 00:00:00.052

HNI_id,name,total_hni_pay
1,Juana Crider,30014918
2,Darrion Matlock,50025008
3,Britney Redmond,30027761
4,Tristin Morse,20015599
5,Annalise Fleming,90023574


2) Asset Management: Asset Managers along with charging fees for their services, also get a profit share from the services they provide. The revenue will be the sum of the fees and profit share. Calculate how much each AM client has paid to the firm. Also show the names of the clients.

In [2]:
with total_pay as (
select AMclient_id, sum(profitshare) + sum(fee) as
total_amclient_pay from dma.AMprovision group by AMclient_id)
select tp.AMclient_id, c.name, tp.total_amclient_pay from
total_pay tp join dma.client c on tp.AMclient_id = c.ID;

(5 rows affected)

Total execution time: 00:00:00.131

AMclient_id,name,total_amclient_pay
6,Connor Champagne,5877
7,Janie Gilman,7367
8,Rashad Cerda,5055
27,Embedly,6330
28,Oyo,8496


3) Investment Banking: Investment Bankers underwrite securities of corporate companies and sell their stocks. They charge a fee/share for underwriting and selling. The revenue brought in by this division will be the (selling price - cost price)\* quantity. Assuming all the securities are sold, show the company name, its net revenue, total selling price, total cost price and the profit Investment bankers make on the sale. The total selling price would be (the cost price per share + the underwriting fee per share) \* quantity

In [3]:
with IB_stock_sell_price as ( select s.security_id,
s.corporateCo_id, (s.price_per_share + cast(u.fee_per_share as
int))*s.total_quantity as sell_price
from dma.securities s join dma.underwriting u on s.security_id =
u.security_id)
select sp.corporateCo_id, c.name, co.net_revenue, sp.sell_price,
(s.price_per_share* cast(s.total_quantity as int)) as cost_price,
(sp.sell_price - (s.price_per_share* cast(s.total_quantity as
int))) as total_profit
from IB_stock_sell_price sp join dma.client c on sp.corporateCo_id
= c.ID join dma.securities s on sp.security_id = s.security_id
join dma.corporateCo co on c.ID = co.client_id;

(5 rows affected)

Total execution time: 00:00:00.044

corporateCo_id,name,net_revenue,sell_price,cost_price,total_profit
26,CustomMade,82000000,1530000,1500000,30000
27,Embedly,62000000,3075000,3000000,75000
28,Oyo,42000000,8040000,8000000,40000
29,Flipkart,22000000,985600,980000,5600
30,Boston Dynamics,20000000,1442400,1440000,2400


4) Private Banking: The money this division brings in is simple. It is the sum of the total savings of the commoners who open an account with the bank. Of course, the net revenue or total profit would be from the high interest loan rates and low savings interest rates. This is captured in the revenue table, but for the sake of the simplicity of this project, this is not expanded.

In [4]:
select sum(total_savings) as total_inflow from dma.commoner;

(1 row affected)

Total execution time: 00:00:00.039

total_inflow
951009


5) The firm wants to look at the running total of the revenue of each of its division starting from 1st jan 2020 to till date to keep track of its progress. Below query calculates the running total ordered by month.

In [5]:
select div_id, month, revenue, sum(revenue)
over (partition by div_id order by month rows between unbounded
preceding and current row) as runningtotal
from dma.revenue;

(144 rows affected)

Total execution time: 00:00:00.050

div_id,month,revenue,runningtotal
1,2020-01-01,1000000000,1000000000
1,2020-02-01,1100000000,2100000000
1,2020-03-01,1200000000,3300000000
1,2020-04-01,1300000000,4600000000
1,2020-05-01,1400000000,6000000000
1,2020-06-01,1500000000,7500000000
1,2020-07-01,1600000000,9100000000
1,2020-08-01,1700000000,10800000000
1,2020-09-01,1800000000,12600000000
1,2020-10-01,1900000000,14500000000


6) Identify the total quantity of securities that can be sold now by the commoners i.e. release date should be before today's date. The purpose is to make arrangements for the sale.

In [6]:
select sp.security_id, sum(quantity) ready_to_sell_quantity
from dma.security_purchase sp
where DATEDIFF(day, CAST( GETDATE() AS Date ), release_date) > 0
group by sp.security_id;

(4 rows affected)

Total execution time: 00:00:00.047

security_id,ready_to_sell_quantity
100001,780
200002,325
300003,184
500005,184


7) What are the mean, min, max and range of salaries of employees in each division? T Display the name of the division and the number of employees in each division. he purpose of this query is for the HR department to compare its compensations with other firms to be able to hire best talent.

In [7]:
select d.NAME, avg(e.compensation) as mean_salary,
min(e.compensation) as min_salary, max(e.compensation) as
max_salary, max(e.compensation) - min(e.compensation) as range,
count(*) as num_of_employees
from dma.employee e join dma.division d on e.div_id=d.ID
GROUP by d.NAME;

(4 rows affected)

Total execution time: 00:00:00.035

NAME,mean_salary,min_salary,max_salary,range,num_of_employees
Asset Management,188527,115753,278131,162378,7
Investment Banking,209513,102294,300000,197706,8
Private and Corporate,208986,114220,298792,184572,8
Wealth Management,223477,120715,300000,179285,7


8) Identify how many employees work for each manager. Display the name of the manager, age, salary and the count. The purpose is to identify if the managers are being compensated properly or not.

In [8]:
with emp_count as (
select manager_id, count(*) as no_of_employees from dma.employee
group by manager_id except (select manager_id, count(*) from
dma.employee
where manager_id is null group by manager_id))
select e.ID, e.name, e.compensation, c.no_of_employees from
dma.employee e join emp_count c on e.ID = c.manager_id;

(4 rows affected)

Total execution time: 00:00:00.040

ID,name,compensation,no_of_employees
4,Venkat Krishnamurthy,300000,7
5,Bridgett Kovach,114220,7
7,Lexis Lofton,270000,6
10,Daniela Ceja,300000,6


9) Identify the commoner IDs of those commoners who availed all PCServices. The purpose of this analytical query is to list those commoners to provide them loyalty benefits.

In [9]:
select commoner_id from dma.PCprovision pc1 where
(select count(distinct pc_service_id)
from dma.PCprovision pc2
where pc1.commoner_id=pc2.commoner_id) =
(select distinct count(*) from dma.PCservice)
group by commoner_id;

(2 rows affected)

Total execution time: 00:00:00.039

commoner_id
11
12
